In [1]:
import zarr
from cellpose import models
from tifffile import imread
import dask.array as da
import napari
import torch
torch.cuda.is_available()



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.11.13 
torch version:  	2.7.1+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




True

In [2]:
file_path = r'R:\CellDive\NSR7649\NSR7649_Final\NSR7649_1.0.4_R000_DAPI__FINAL_F.ome.tif'

In [3]:
store = imread(file_path, aszarr=True)
root = zarr.open(store, mode="r") 
im_list = []
for i in range(5):
    im = da.from_zarr(root[str(i)])
    im_list.append(im)

im_list[0]

dask.array<from-zarr, shape=(68754, 92415), dtype=uint16, chunksize=(512, 512), chunktype=numpy.ndarray>

In [ ]:
model = models.CellposeModel(gpu=True)

flow_threshold = 0.4
cellprob_threshold = -0.2
tile_norm_blocksize = 0

im_small = im_list[0]#[29000:30000, 45000:46000]

o1, o2, o3 = model.eval(im_small, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold,
                                normalize={"tile_norm_blocksize": tile_norm_blocksize})


In [8]:
viewer = napari.Viewer()
viewer.add_image(im_list)
viewer.add_labels(o1,translate=(29000, 45000))

<Labels layer 'o1' at 0x1de3c243dd0>

In [14]:
from instanseg import InstanSeg
import numpy as np

In [10]:
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1)

Model fluorescence_nuclei_and_cells version 0.1.0 downloaded and extracted to c:\Users\KMK280\AppData\Local\miniconda3\envs\samcellpose-env\Lib\site-packages\instanseg\utils\../bioimageio_models/
Requesting default device: cuda


In [36]:
im

dask.array<from-zarr, shape=(4297, 5775), dtype=uint16, chunksize=(512, 512), chunktype=numpy.ndarray>

In [ ]:
pixel_size = 0.4
im_test = np.stack([im_small.compute(),im_small.compute()], axis=-1)
labeled_output, _ = instanseg_fluorescence.eval_small_image(im_test, 
                                                        pixel_size, 
                                                        resolve_cell_and_nucleus=False, 
                                                        mean_threshold = 0.3, 
                                                        cleanup_fragments = True)

segm_array = [np.array(x).astype(int) for x in labeled_output[0, :, :, :]]

In [37]:
im = im_list[0]

pixel_size = 0.4
im_test = np.stack([im,im, im], axis=-1)
im_test = im_test[29000:30000, 45000:46000,:].compute()
labeled_output, _ = instanseg_fluorescence.eval_small_image(im_test,
                                                    pixel_size,
                                                    resolve_cell_and_nucleus=False,
                                                    mean_threshold = 0.3,
                                                    cleanup_fragments = True)

segm_array = [np.array(x).astype(int) for x in labeled_output[0, :, :, :]]

In [38]:
im_test.shape

(1000, 1000, 3)

In [30]:
viewer.add_labels(segm_array[0],translate=(29000, 45000))
viewer.add_labels(segm_array[1],translate=(29000, 45000))

<Labels layer 'Labels [1]' at 0x1df009c9190>

In [4]:
mask_path = r'D:\mask_todel\test_30_30.tif'
mask = imread(mask_path)